## Baseline V2 - 2024.01.20

In [1]:
from src.models.classification.models import Model
from src.base import BasePiepline
# from src.utils.data_preprocessing import DataPreprocessing
from src.utils.manage_pkl_files import get_best_params
from src.utils.set_seed import seed_everything
from sklearn.preprocessing import OneHotEncoder

SEED = 42 # NOTE: 시드 값 설정
seed_everything(SEED)

model = Model()

### 전처리 클래스 상속 예시

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings(action='ignore')

class DataPreprocessing:
    def __init__(self):        
        pass

    def set_up_dataframe(self,df,mode):
        self.df = df # df에 새롭게 저장
        self.mode = mode
        if self.mode:
            self.label_encoder_ = {} # data leakage 방지
        
    def init(self):
        """ 전처리 초기 """
        self.df = self.df.drop(columns=['ID'])


        # 근로기간도 따로 빼내야 하나?
        label_dict = {'< 1 year': 0, '<1 year': 0, '1 year': 1, '1 years': 1, '2 years': 2, '3': 3,
                      '3 years': 3, '4 years': 4, '5 years': 5, '6 years': 6, '7 years': 7, '8 years': 8,
                      '9 years': 9, '10+ years': 10, '10+years': 10, 'Unknown': -1
                      }

        self.df['근로기간'] = self.df['근로기간'].map(label_dict)

    def categorical_to_numeric(self):
        """ 범주형 수치형으로 변환 """
        categorical_features = ['대출기간', '주택소유상태']
        for idx in categorical_features:
            if self.mode: # train, valid이면
                self.label_encoder_[idx] = LabelEncoder() #딕셔너리에 저장
                self.df[idx] = self.label_encoder_[idx].fit_transform(self.df[idx])
            else: # test이면
                self.df[idx] = self.label_encoder_[idx].transform(self.df[idx])
                    
                

    def repayment_month_interest_rate(self):
        """
        강지원 선생님 과 고건영 제자의 작품
        상환개월 및 이자율 피쳐 생성
        """
        self.df["갚은금액"] = self.df["총상환원금"] + self.df["총상환이자"]
        # 가정 1
        conv1 = self.df["총연체금액"] == 0 
        # 가정 2
        conv2 = self.df["갚은금액"] > 0  
        data = self.df[conv1 & conv2]
        # 월원금상환금 피쳐 생성
        data['대출기간'] = data['대출기간'].astype(str).str[:3].astype(int) # 말이 됨
        data["월원금상환금"] = data["대출금액"] / data["대출기간"]
        # 상환개월 피쳐 생성
        data["상환개월"] = round(data["총상환원금"] / data["월원금상환금"])
        # 이자율 피쳐 생성
        data["이자율"] = data["총상환이자"] / (data['대출금액'] * data['상환개월'])
        # null값 변경
        data = data.replace([np.inf, -np.inf], np.nan)
        # 결측치 채우기
        if self.mode:
            self.rate_mean = data["이자율"].mean() # 인스턴스변수에 저장 data leakage 해소 -> 나중에 저장하는 딕셔너리 변수로 만들어도 좋을 듯?
        data["이자율"].fillna(self.rate_mean, inplace = True)
        # 원본 데이터에 복원
        self.df["상환개월"] = data["상환개월"]
        self.df["이자율"] = data["이자율"]
        # 이자율 결측치 채우기 (연체 한 사람들)
        self.df["이자율"].fillna(self.rate_mean, inplace = True)
        # 갚은 금액 0인 사람들 상환개월 0으로 채우기
        conv3 = self.df["갚은금액"] == 0
        li = self.df[conv3].index.to_list()
        self.df.loc[li, '상환개월'] = 0
        # 연체한 사람들 상환 개월 구하기
        bb = self.df[self.df['상환개월'].isna()]
        bb["월상환이자"] = bb["대출금액"] * bb['이자율']
        bb['대출기간'] = bb['대출기간'].astype(str).str[:3].astype(int) # 말이 됨
        bb["월원금상환금"] = bb["대출금액"] / bb["대출기간"]
        bb["상환개월"] = round(bb["총상환이자"] / bb["월상환이자"])
        self.df.loc[self.df[self.df["상환개월"].isna()].index.to_list(), "상환개월"] = bb["상환개월"]

    def etcetera(self):
        """ 이상한 값들 변환 전처리 """
        self.df[self.df["주택소유상태"] != "ANY"].reset_index(drop=True, inplace=True)
        
    def loan_object(self):
        """ 대출 목적 변환 """
        train_object = self.df['대출목적'].unique()  # NOTE: not used variable
        oject_dict = dict()
        for i, v in enumerate(self.df['대출목적'].unique()):
            oject_dict[v] = i

        self.df['대출목적'] = self.df['대출목적'].apply(lambda x: oject_dict.get(x, '기타'))

    def remaining_loan_amount(self):
        """ 남은대출금액 피쳐 생성 """
        self.df['연간소득'] = pd.qcut(self.df['연간소득'], q=10, labels=False)
        self.df["총상환금"] = self.df["총상환원금"] + self.df["총상환이자"]
        self.df["남은대출금액"] = self.df["대출금액"] - self.df["총상환금"]
        self.df = self.df.drop(columns=['총상환금'])
        
    def drop_column(self):
        self.df = self.df.drop(columns=['갚은금액'])
        
    def run(self):
        # 순서나 뺄거등 개인이 조절
        self.init()
        self.repayment_month_interest_rate()
        self.etcetera()
        self.loan_object()
        self.remaining_loan_amount()
        self.drop_column()
        self.categorical_to_numeric()
        return self.df

### 분류 모델 클래스 및 파이프라인 선언

In [3]:
class ModelProcess(BasePiepline):
    def __init__(self, preprocess_object):
        super().__init__()
        self.preprocessing = preprocess_object()
model_process = ModelProcess(DataPreprocessing) # NOTE: 커스텀 전처리 클래스를 사용할 경우

### 최적화 파라미터 값 가져오기

In [4]:
params = get_best_params()

### 모델 파라미터 적용

In [7]:
_model = model.xgboost.set_params(**params)

### 스코어 확인

In [8]:
model_process.valid(_model)

5it [02:25, 29.01s/it]

----[K-Fold Validation Score]-----
f1 score : 0.8416 / STD: (+/- 0.0024)
precision score : 0.8313 / STD: (+/- 0.0022)
recall score : 0.8533 / STD: (+/- 0.0036)


### 전체 실행(서브밋 단계 까지)

In [18]:
model_process.run(_model)

진입
0.012153916925365852
{'model_name': 'XGBClassifier', 'model_instance': XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8734150979529018, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.017600210882064204,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.0947263260073529,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=373, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)}
0.012153916925365852
?여ㅑ기인가?
